In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

In [2]:
train = pd.read_csv('../../data/train_2016_v2.csv',parse_dates=['transactiondate'])
prop = pd.read_csv('../../data/properties_2016.csv')
sample = pd.read_csv('../../data/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
sample.drop('ParcelId',axis = 1,inplace= True)

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

In [4]:
df_train = train.merge(prop, how='left', on='parcelid')
df_test = sample.merge(prop, how='left', on='parcelid')

In [5]:
df_parcelid_count = df_train['parcelid'].value_counts().reset_index()
df_parcelid_count.columns = ['parcelid','count']
print(df_parcelid_count['count'].value_counts())
del df_parcelid_count; gc.collect()
## most of parcelid is unique, that is to say few properties(124 parcelid) were sold multiple times during 2016

1    90026
2      123
3        1
Name: count, dtype: int64


7

In [6]:
parcelid_train = np.unique(df_train['parcelid'])
print(len(df_train))
print(len(df_test))
print(len(df_test[df_test['parcelid'].isin(parcelid_train) == False]))
del parcelid_train; gc.collect()
## all of parcelid showed in train need to be predicted in time point 201610 201611 201612
## the others are unkown

90275
2985217
2895067


0

In [7]:
import datetime
def GenerateLastGap(data):
    result = pd.Series(index = data.index)
    for i in data.index:
        if((i == 0) or (data.at[i,'parcelid'] != data.at[i - 1,'parcelid'])):
            result[i] = 0
        else:
            result[i] = data.at[i,'transactiondate'].month - data.at[i - 1,'transactiondate'].month
    return result
## last gap for train
df_train_lastgap = df_train.sort_values(['parcelid','transactiondate'],inplace= False)
df_train_lastgap['idx'] = range(len(df_train_lastgap))
df_tmp = df_train_lastgap[['idx','parcelid','transactiondate']].set_index(['idx'])
df_lastgap = pd.DataFrame(GenerateLastGap(df_tmp),columns= ['lastgap']).reset_index()
df_train_lastgap = df_train_lastgap.merge(df_lastgap,on='idx',how='left')
df_train_lastgap.drop(['idx'],inplace=True,axis = 1)
df_train = df_train_lastgap.copy()
del df_train_lastgap,df_tmp,df_lastgap; gc.collect()

120

In [8]:
## last gap for test
df_latest_transaction = df_train.groupby(['parcelid'])['transactiondate'].max().to_frame().reset_index()
df_latest_transaction.columns = ['parcelid','lasttransactiondate']
df_test_lastgap = df_test.merge(df_latest_transaction,on = 'parcelid',how = 'left')
l_test_transdate = ['201610','201611','201612','201710','201711','201712']
for d in l_test_transdate:
    mt = datetime.datetime.strptime(d,'%Y%m').month
    df_test_lastgap['lastgap%s' % d] = df_test_lastgap['lasttransactiondate'].apply(lambda v: 0 if(pd.isnull(v)) else (mt - v.month))
df_test_lastgap[df_test_lastgap['lastgap201610'] > 0].head(20)
df_test = df_test_lastgap.copy()
del df_latest_transaction,df_test_lastgap;gc.collect()
#df_train_lastgap[df_train_lastgap['lastgap'] > 0]['parcelid']
#print(df_train_lastgap[df_train_lastgap['parcelid'] == 10799924])
#print(tmp.ix[10799924])

41

In [9]:
missing_df = df_train.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / df_train.shape[0]
missing_df.ix[missing_df['missing_ratio']>0.995]

,column_name,missing_count,missing_ratio
4,architecturalstyletypeid,90014,0.997109
5,basementsqft,90232,0.999524
8,buildingclasstypeid,90259,0.999823
15,finishedsquarefeet13,90242,0.999634
18,finishedsquarefeet6,89854,0.995336
43,storytypeid,90232,0.999524
45,typeconstructiontypeid,89976,0.996688
48,yardbuildingsqft26,90180,0.998948
51,fireplaceflag,90053,0.997541


In [93]:
ObjectColumns = []
for c, dtype in zip(df_train.columns, df_train.dtypes):
    if dtype == np.float64:
        df_train[c] = df_train[c].astype(np.float32)
    elif dtype == np.object:
        ObjectColumns.append(c)
df_train.dtypes.value_counts()
print(ObjectColumns)

['hashottuborspa', 'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag']


In [94]:
#hashottuborspa
NewCols = [col for col in df_train['hashottuborspa'].value_counts().keys()]
for k in NewCols:
    df_train['hashottuborspa-%s' % k] = 0
    df_train.loc[df_train['hashottuborspa'] == k,'hashottuborspa-%s' % k] = 1
df_train['hashottuborspa-Missing'] = 0
df_train.loc[df_train['hashottuborspa'].isnull() == True,'hashottuborspa-Missing'] = 1
df_train[[col  for col in df_train.columns if 'hashottuborspa' in col]]['hashottuborspa-True'].value_counts()
df_train = df_train.drop(['hashottuborspa'],axis= 1)

In [95]:
# fireplaceflag
print(len(df_train))
NewCols = [col for col in df_train['fireplaceflag'].value_counts().keys()]
for k in NewCols:
    df_train['fireplaceflag-%s' % k] = 0
    df_train.loc[df_train['fireplaceflag'] == k,'fireplaceflag-%s' % k] = 1
df_train['fireplaceflag-Missing'] = 0
df_train.loc[df_train['fireplaceflag'].isnull() == True,'fireplaceflag-Missing'] = 1
df_train[[col  for col in df_train.columns if 'fireplaceflag' in col]]['fireplaceflag-True'].value_counts()
df_train = df_train.drop(['fireplaceflag'],axis= 1)

90275


In [101]:
# taxdelinquencyflag
NewCols = [col for col in df_train['taxdelinquencyflag'].value_counts().keys()]
for k in NewCols:
    df_train['taxdelinquencyflag-%s' % k] = 0
    df_train.loc[df_train['taxdelinquencyflag'] == k,'taxdelinquencyflag-%s' % k] = 1
df_train['taxdelinquencyflag-Missing'] = 0
df_train.loc[df_train['taxdelinquencyflag'].isnull() == True,'taxdelinquencyflag-Missing'] = 1
df_train[[col  for col in df_train.columns if 'taxdelinquencyflag' in col]]['taxdelinquencyflag-Y'].value_counts()
df_train = df_train.drop(['taxdelinquencyflag'],axis= 1)

In [104]:
OrdinalCols = []
for c, dtype in zip(df_train.columns, df_train.dtypes):
    if dtype == np.float32:
        OrdinalCols.append(c)
print(OrdinalCols)

['logerror', 'airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt', 'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'taxdelinquencyye

In [116]:
# airconditioningtypeid
print(len(df_train))
print(df_train['airconditioningtypeid'].value_counts())
NewCols = [col for col in df_train['airconditioningtypeid'].value_counts().keys()]
for k in NewCols:
    df_train['airconditioningtypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['airconditioningtypeid'] == k,'airconditioningtypeid-%s' % str(int(k))] = 1
df_train['airconditioningtypeid-Missing'] = 0
df_train.loc[df_train['airconditioningtypeid'].isnull() == True,'airconditioningtypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'airconditioningtypeid' in col]]['airconditioningtypeid-13'].value_counts()
df_train = df_train.drop(['airconditioningtypeid'],axis= 1)

90275
1.0     26668
13.0     1833
5.0       215
11.0       63
9.0         1
3.0         1
Name: airconditioningtypeid, dtype: int64


In [123]:
# architecturalstyletypeid
print(len(df_train))
print(df_train['architecturalstyletypeid'].value_counts())
NewCols = [col for col in df_train['architecturalstyletypeid'].value_counts().keys()]
for k in NewCols:
    df_train['architecturalstyletypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['architecturalstyletypeid'] == k,'architecturalstyletypeid-%s' % str(int(k))] = 1
df_train['architecturalstyletypeid-Missing'] = 0
df_train.loc[df_train['architecturalstyletypeid'].isnull() == True,'architecturalstyletypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'architecturalstyletypeid' in col]]['airconditioningtypeid-13'].value_counts()
df_train = df_train.drop(['airconditioningtypeid'],axis= 1)

90275


KeyError: 'architecturalstyletypeid'

In [126]:
# buildingqualitytypeid
print(len(df_train))
print(df_train['buildingqualitytypeid'].value_counts())
NewCols = [col for col in df_train['buildingqualitytypeid'].value_counts().keys()]
for k in NewCols:
    df_train['buildingqualitytypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['buildingqualitytypeid'] == k,'buildingqualitytypeid-%s' % str(int(k))] = 1
df_train['buildingqualitytypeid-Missing'] = 0
df_train.loc[df_train['buildingqualitytypeid'].isnull() == True,'buildingqualitytypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'buildingqualitytypeid' in col]]['buildingqualitytypeid-8'].value_counts()
df_train = df_train.drop(['buildingqualitytypeid'],axis= 1)

90275
7.0     29310
4.0     23839
1.0      2627
10.0     1461
12.0      119
8.0         5
6.0         2
11.0        1
Name: buildingqualitytypeid, dtype: int64


In [130]:
# decktypeid
print(len(df_train))
print(df_train['decktypeid'].value_counts())
NewCols = [col for col in df_train['decktypeid'].value_counts().keys()]
for k in NewCols:
    df_train['decktypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['decktypeid'] == k,'decktypeid-%s' % str(int(k))] = 1
df_train['decktypeid-Missing'] = 0
df_train.loc[df_train['decktypeid'].isnull() == True,'decktypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'decktypeid' in col]]['decktypeid-66'].value_counts()
df_train = df_train.drop(['decktypeid'],axis= 1)

90275
66.0    658
Name: decktypeid, dtype: int64


In [134]:
# fips
print(len(df_train))
print(df_train['fips'].value_counts())
NewCols = [col for col in df_train['fips'].value_counts().keys()]
for k in NewCols:
    df_train['fips-%s' % str(int(k))] = 0
    df_train.loc[df_train['fips'] == k,'fips-%s' % str(int(k))] = 1
df_train['fips-Missing'] = 0
df_train.loc[df_train['fips'].isnull() == True,'fips-Missing'] = 1
df_train[[col  for col in df_train.columns if 'fips' in col]]['fips-6059'].value_counts()
df_train = df_train.drop(['fips'],axis= 1)

90275
6037.0    58574
6059.0    24505
6111.0     7196
Name: fips, dtype: int64


In [138]:
# heatingorsystemtypeid
print(len(df_train))
print(df_train['heatingorsystemtypeid'].value_counts())
NewCols = [col for col in df_train['heatingorsystemtypeid'].value_counts().keys()]
for k in NewCols:
    df_train['heatingorsystemtypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['heatingorsystemtypeid'] == k,'heatingorsystemtypeid-%s' % str(int(k))] = 1
df_train['heatingorsystemtypeid-Missing'] = 0
df_train.loc[df_train['heatingorsystemtypeid'].isnull() == True,'heatingorsystemtypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'heatingorsystemtypeid' in col]]['heatingorsystemtypeid-7'].value_counts()
df_train = df_train.drop(['heatingorsystemtypeid'],axis= 1)

90275
2.0     38303
7.0     15519
24.0     1071
6.0       970
20.0       97
13.0       76
18.0       25
1.0        13
14.0        2
10.0        2
12.0        1
11.0        1
Name: heatingorsystemtypeid, dtype: int64


In [142]:
# pooltypeid10
print(len(df_train))
print(df_train['pooltypeid10'].value_counts())
NewCols = [col for col in df_train['pooltypeid10'].value_counts().keys()]
for k in NewCols:
    df_train['pooltypeid10-%s' % str(int(k))] = 0
    df_train.loc[df_train['pooltypeid10'] == k,'pooltypeid10-%s' % str(int(k))] = 1
df_train['pooltypeid10-Missing'] = 0
df_train.loc[df_train['pooltypeid10'].isnull() == True,'pooltypeid10-Missing'] = 1
df_train[[col  for col in df_train.columns if 'pooltypeid10' in col]]['pooltypeid10-1'].value_counts()
df_train = df_train.drop(['pooltypeid10'],axis= 1)

90275
1.0    1161
Name: pooltypeid10, dtype: int64


In [144]:
# pooltypeid2
print(len(df_train))
print(df_train['pooltypeid2'].value_counts())
NewCols = [col for col in df_train['pooltypeid2'].value_counts().keys()]
for k in NewCols:
    df_train['pooltypeid2-%s' % str(int(k))] = 0
    df_train.loc[df_train['pooltypeid2'] == k,'pooltypeid2-%s' % str(int(k))] = 1
df_train['pooltypeid2-Missing'] = 0
df_train.loc[df_train['pooltypeid2'].isnull() == True,'pooltypeid2-Missing'] = 1
df_train[[col  for col in df_train.columns if 'pooltypeid2' in col]]['pooltypeid2-1'].value_counts()
df_train = df_train.drop(['pooltypeid2'],axis= 1)

90275
1.0    1204
Name: pooltypeid2, dtype: int64


In [145]:
# pooltypeid7
print(len(df_train))
print(df_train['pooltypeid7'].value_counts())
NewCols = [col for col in df_train['pooltypeid7'].value_counts().keys()]
for k in NewCols:
    df_train['pooltypeid7-%s' % str(int(k))] = 0
    df_train.loc[df_train['pooltypeid7'] == k,'pooltypeid7-%s' % str(int(k))] = 1
df_train['pooltypeid7-Missing'] = 0
df_train.loc[df_train['pooltypeid7'].isnull() == True,'pooltypeid7-Missing'] = 1
df_train[[col  for col in df_train.columns if 'pooltypeid7' in col]]['pooltypeid7-1'].value_counts()
df_train = df_train.drop(['pooltypeid7'],axis= 1)

90275
1.0    16697
Name: pooltypeid7, dtype: int64


In [149]:
# propertylandusetypeid
print(len(df_train))
print(df_train['propertylandusetypeid'].value_counts())
NewCols = [col for col in df_train['propertylandusetypeid'].value_counts().keys()]
for k in NewCols:
    df_train['propertylandusetypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['propertylandusetypeid'] == k,'propertylandusetypeid-%s' % str(int(k))] = 1
df_train['propertylandusetypeid-Missing'] = 0
df_train.loc[df_train['propertylandusetypeid'].isnull() == True,'propertylandusetypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'propertylandusetypeid' in col]]['propertylandusetypeid-269'].value_counts()
df_train = df_train.drop(['propertylandusetypeid'],axis= 1)

90275
261.0    60637
266.0    22815
246.0     2376
269.0     2334
248.0      879
247.0      629
265.0      356
263.0       84
260.0       62
275.0       46
267.0       28
31.0        17
264.0       11
47.0         1
Name: propertylandusetypeid, dtype: int64


In [155]:
# rawcensustractandblock
print(len(df_train))
print(df_train['rawcensustractandblock'].value_counts())
NewCols = [col for col in df_train['rawcensustractandblock'].value_counts().keys()]
for k in NewCols:
    df_train['rawcensustractandblock-%s' % str(int(k))] = 0
    df_train.loc[df_train['rawcensustractandblock'] == k,'rawcensustractandblock-%s' % str(int(k))] = 1
df_train['rawcensustractandblock-Missing'] = 0
df_train.loc[df_train['rawcensustractandblock'].isnull() == True,'rawcensustractandblock-Missing'] = 1
df_train[[col  for col in df_train.columns if 'rawcensustractandblock' in col]]['rawcensustractandblock-60378004'].value_counts()
df_train = df_train.drop(['rawcensustractandblock'],axis= 1)

90275
60590320.0    3297
60590628.0    2665
60590524.0    2404
60379200.0    2201
60590424.0    2113
60590220.0    1763
60590992.0    1405
61110076.0    1334
60379108.0    1126
60379104.0    1010
60378004.0     980
60590756.0     937
60590996.0     919
60379012.0     880
60590016.0     856
60591100.0     833
60379204.0     832
61110052.0     758
60374032.0     758
60379008.0     701
60590116.0     597
61110060.0     594
60590760.0     559
60590632.0     528
60376704.0     498
60590420.0     494
61110084.0     488
60374004.0     473
60371396.0     470
60376212.0     440
              ... 
60371840.0      19
60371904.0      19
60372240.0      18
60372228.0      16
60372100.0      16
60372000.0      15
60371880.0      15
60379304.0      15
60372032.0      14
60372268.0      13
60372096.0      12
60375756.0      11
60372136.0      11
60371928.0      11
60372416.0      10
60376100.0      10
60371996.0      10
60372280.0       9
60372144.0       9
60372092.0       9
60379800.0       7
603722

In [160]:
# regionidcity
print(len(df_train))
print(df_train['regionidcity'].value_counts())
NewCols = [col for col in df_train['regionidcity'].value_counts().keys()]
for k in NewCols:
    df_train['regionidcity-%s' % str(int(k))] = 0
    df_train.loc[df_train['regionidcity'] == k,'regionidcity-%s' % str(int(k))] = 1
df_train['regionidcity-Missing'] = 0
df_train.loc[df_train['regionidcity'].isnull() == True,'regionidcity-Missing'] = 1
df_train[[col  for col in df_train.columns if 'regionidcity' in col]]['regionidcity-47568'].value_counts()
df_train = df_train.drop(['regionidcity'],axis= 1)

90275
12447.0     20559
46298.0      3000
52650.0      2132
54311.0      2128
5534.0       2035
40227.0      1949
16764.0      1800
25218.0      1586
34278.0      1383
27110.0      1266
12773.0      1222
47019.0      1187
13150.0      1120
47568.0      1097
45457.0      1052
24812.0      1037
33252.0      1034
34543.0       949
51239.0       932
54722.0       929
53571.0       923
25459.0       904
24832.0       891
32380.0       820
13693.0       815
37086.0       791
33612.0       772
20008.0       756
21412.0       729
15554.0       693
            ...  
21778.0        50
30399.0        49
113412.0       47
118875.0       40
16961.0        39
16389.0        36
272578.0       33
32753.0        30
3491.0         30
25468.0        27
38980.0        25
6822.0         24
13232.0        21
42091.0        20
114834.0       18
34037.0        16
25271.0        14
33312.0        14
25621.0        14
14906.0        13
31134.0        12
53162.0         9
32927.0         5
10815.0         4
1188

In [164]:
# regionidcounty
print(len(df_train))
print(df_train['regionidcounty'].value_counts())
NewCols = [col for col in df_train['regionidcounty'].value_counts().keys()]
for k in NewCols:
    df_train['regionidcounty-%s' % str(int(k))] = 0
    df_train.loc[df_train['regionidcounty'] == k,'regionidcounty-%s' % str(int(k))] = 1
df_train['regionidcounty-Missing'] = 0
df_train.loc[df_train['regionidcounty'].isnull() == True,'regionidcounty-Missing'] = 1
df_train[[col  for col in df_train.columns if 'regionidcounty' in col]]['regionidcounty-1286'].value_counts()
df_train = df_train.drop(['regionidcounty'],axis= 1)

90275
3101.0    58574
1286.0    24505
2061.0     7196
Name: regionidcounty, dtype: int64


In [168]:
# regionidneighborhood
print(len(df_train))
print(df_train['regionidneighborhood'].value_counts())
NewCols = [col for col in df_train['regionidneighborhood'].value_counts().keys()]
for k in NewCols:
    df_train['regionidneighborhood-%s' % str(int(k))] = 0
    df_train.loc[df_train['regionidneighborhood'] == k,'regionidneighborhood-%s' % str(int(k))] = 1
df_train['regionidneighborhood-Missing'] = 0
df_train.loc[df_train['regionidneighborhood'].isnull() == True,'regionidneighborhood-Missing'] = 1
df_train[[col  for col in df_train.columns if 'regionidneighborhood' in col]]['regionidneighborhood-274049'].value_counts()
df_train = df_train.drop(['regionidneighborhood'],axis= 1)

90275
118208.0    773
27080.0     755
48570.0     753
37739.0     715
48200.0     653
268496.0    631
51906.0     609
54300.0     540
113455.0    529
33183.0     497
274049.0    479
34213.0     477
46736.0     454
47880.0     450
47950.0     428
31817.0     417
113910.0    409
6952.0      408
274514.0    403
40548.0     397
118920.0    391
41131.0     371
268588.0    370
26134.0     352
276476.0    352
21056.0     341
41466.0     334
19810.0     332
32059.0     308
27987.0     299
           ... 
761214.0      1
273607.0      1
260382.0      1
761098.0      1
763217.0      1
761090.0      1
763791.0      1
764140.0      1
416332.0      1
272732.0      1
762937.0      1
275287.0      1
273313.0      1
762932.0      1
762929.0      1
762927.0      1
764138.0      1
764136.0      1
762947.0      1
268446.0      1
762949.0      1
416335.0      1
762955.0      1
764152.0      1
762960.0      1
764165.0      1
762191.0      1
272170.0      1
764139.0      1
762963.0      1
Name: regionidneig

In [172]:
# regionidzip
print(len(df_train))
print(df_train['regionidzip'].value_counts())
NewCols = [col for col in df_train['regionidzip'].value_counts().keys()]
for k in NewCols:
    df_train['regionidzip-%s' % str(int(k))] = 0
    df_train.loc[df_train['regionidzip'] == k,'regionidzip-%s' % str(int(k))] = 1
df_train['regionidzip-Missing'] = 0
df_train.loc[df_train['regionidzip'].isnull() == True,'regionidzip-Missing'] = 1
df_train[[col  for col in df_train.columns if 'regionidzip' in col]]['regionidzip-97329'].value_counts()
df_train = df_train.drop(['regionidzip'],axis= 1)

90275
97319.0     912
96987.0     902
96974.0     824
97329.0     760
97318.0     726
97118.0     719
97328.0     704
96993.0     687
96996.0     640
96962.0     624
96193.0     593
96995.0     570
96368.0     560
97116.0     546
96385.0     521
96964.0     516
96998.0     513
96505.0     512
96954.0     511
96989.0     504
96373.0     500
97091.0     498
96186.0     494
96377.0     493
96364.0     476
96389.0     474
97078.0     474
97083.0     464
96351.0     456
96236.0     449
           ... 
96038.0      37
96986.0      35
96135.0      35
96323.0      33
96980.0      33
96207.0      32
97037.0      32
96119.0      30
96434.0      30
96021.0      29
96973.0      27
96951.0      26
97316.0      26
95998.0      21
97331.0      20
95991.0      18
97119.0      17
97324.0      16
399675.0     13
95995.0      12
96148.0      11
97108.0       6
96329.0       5
96002.0       4
96039.0       3
96500.0       2
96034.0       1
97111.0       1
96467.0       1
96226.0       1
Name: regionidzip,

In [176]:
# typeconstructiontypeid
print(len(df_train))
print(df_train['typeconstructiontypeid'].value_counts())
NewCols = [col for col in df_train['typeconstructiontypeid'].value_counts().keys()]
for k in NewCols:
    df_train['typeconstructiontypeid-%s' % str(int(k))] = 0
    df_train.loc[df_train['typeconstructiontypeid'] == k,'typeconstructiontypeid-%s' % str(int(k))] = 1
df_train['typeconstructiontypeid-Missing'] = 0
df_train.loc[df_train['typeconstructiontypeid'].isnull() == True,'typeconstructiontypeid-Missing'] = 1
df_train[[col  for col in df_train.columns if 'typeconstructiontypeid' in col]]['typeconstructiontypeid-4'].value_counts()
df_train = df_train.drop(['typeconstructiontypeid'],axis= 1)

90275
6.0     296
4.0       2
13.0      1
Name: typeconstructiontypeid, dtype: int64


In [181]:
# censustractandblock
print(len(df_train))
print(df_train['censustractandblock'].value_counts())
NewCols = [col for col in df_train['censustractandblock'].value_counts().keys()]
for k in NewCols:
    df_train['censustractandblock-%s' % str(int(k))] = 0
    df_train.loc[df_train['censustractandblock'] == k,'censustractandblock-%s' % str(int(k))] = 1
df_train['censustractandblock-Missing'] = 0
df_train.loc[df_train['censustractandblock'].isnull() == True,'censustractandblock-Missing'] = 1
#df_train[[col  for col in df_train.columns if 'censustractandblock' in col]]['censustractandblock-4'].value_counts()
df_train = df_train.drop(['censustractandblock'],axis= 1)

90275
6.059032e+13    3268
6.059063e+13    2610
6.059053e+13    2345
6.037920e+13    2118
6.059042e+13    1809
6.059022e+13    1753
6.059099e+13    1400
6.059076e+13    1159
6.037911e+13    1112
6.111007e+13    1018
6.037901e+13    1007
6.037910e+13     983
6.059110e+13     945
6.037901e+13     924
6.059100e+13     919
6.037403e+13     905
6.037920e+13     831
6.059042e+13     797
6.111005e+13     780
6.111008e+13     765
6.037800e+13     745
6.111006e+13     646
6.037620e+13     595
6.037670e+13     582
6.059002e+13     576
6.059001e+13     563
6.037409e+13     526
6.059063e+13     513
6.037113e+13     478
6.059076e+13     474
                ... 
6.037292e+13      21
6.037202e+13      21
6.111009e+13      20
6.037200e+13      20
6.037208e+13      20
6.037233e+13      18
6.037222e+13      18
6.037210e+13      15
6.037182e+13      14
6.037203e+13      14
6.037191e+13      14
6.037121e+13      13
6.037227e+13      13
6.037278e+13      12
6.037209e+13      12
6.037277e+13      12
6.03722

In [189]:
print(len(df_train))
print(df_train['yearbuilt'].value_counts())

90275
1955.0    2261
1989.0    2058
1950.0    1994
1954.0    1940
1964.0    1919
1973.0    1818
1956.0    1760
1979.0    1671
1953.0    1661
1987.0    1635
1990.0    1620
1963.0    1541
1980.0    1518
1986.0    1505
1977.0    1489
1951.0    1456
1965.0    1449
1985.0    1435
1952.0    1434
1972.0    1429
1974.0    1385
1978.0    1378
1948.0    1346
1981.0    1324
1947.0    1310
1971.0    1291
1984.0    1282
1988.0    1264
1976.0    1249
1957.0    1239
          ... 
1919.0      99
1907.0      94
1933.0      83
1909.0      79
1906.0      76
1916.0      74
1905.0      69
1918.0      67
1934.0      62
1917.0      56
1901.0      41
1903.0      36
1902.0      26
1904.0      25
2015.0      23
1890.0      18
1895.0      16
1900.0      13
1898.0      10
1899.0       6
1885.0       6
1896.0       5
1894.0       5
1897.0       3
1893.0       3
1892.0       3
1888.0       2
1891.0       1
1887.0       1
1886.0       1
Name: yearbuilt, dtype: int64


In [15]:
x_train = df_train.drop(['parcelid', 
               'logerror', 
               'transactiondate', 
               'propertyzoningdesc', 
               'propertycountylandusecode',
               'basementsqft',
               'buildingclasstypeid',
               'finishedsquarefeet13',
               'storytypeid',
               'assessmentyear'], axis=1,inplace = False)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

# del df_train; gc.collect()

split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.012
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l2'
params['sub_feature'] = 0.8
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 256
params['min_data'] = 500
params['min_hessian'] = 0.05

watchlist = [d_valid]
clf = lgb.train(params, d_train, 470, watchlist)

# del d_train, d_valid; gc.collect()
# del x_train, x_valid; gc.collect()

(90275, 52) (90275,)
[1]	valid_0's l2: 0.0126519
[2]	valid_0's l2: 0.0126524
[3]	valid_0's l2: 0.0126567
[4]	valid_0's l2: 0.0126572
[5]	valid_0's l2: 0.0126568
[6]	valid_0's l2: 0.0126603
[7]	valid_0's l2: 0.0126576
[8]	valid_0's l2: 0.0126571
[9]	valid_0's l2: 0.0126543
[10]	valid_0's l2: 0.0126586
[11]	valid_0's l2: 0.0126602
[12]	valid_0's l2: 0.0126586
[13]	valid_0's l2: 0.0126543
[14]	valid_0's l2: 0.0126476
[15]	valid_0's l2: 0.0126428
[16]	valid_0's l2: 0.0126475
[17]	valid_0's l2: 0.012646
[18]	valid_0's l2: 0.0126485
[19]	valid_0's l2: 0.0126451
[20]	valid_0's l2: 0.0126503
[21]	valid_0's l2: 0.0126534
[22]	valid_0's l2: 0.012654
[23]	valid_0's l2: 0.0126536
[24]	valid_0's l2: 0.0126537
[25]	valid_0's l2: 0.0126517
[26]	valid_0's l2: 0.0126554
[27]	valid_0's l2: 0.0126592
[28]	valid_0's l2: 0.012661
[29]	valid_0's l2: 0.0126592
[30]	valid_0's l2: 0.0126616
[31]	valid_0's l2: 0.0126601
[32]	valid_0's l2: 0.0126619
[33]	valid_0's l2: 0.012664
[34]	valid_0's l2: 0.0126635
[35]	v

In [2]:
print("Start write result ...")
sub = pd.read_csv('../../data/sample_submission.csv')

import pandas ad 
import time

start = time.time()

CalibrationFactor = 0.03
cal = CalibrationFactor * 0.013

for d in l_test_transdate:
    l_test_columns = ['lastgap%s' % d if (c == 'lastgap') else c for c in train_columns]
    x_test = df_test[l_test_columns]
    for c in x_test.dtypes[x_test.dtypes == object].index.values:
        x_test[c] = (x_test[c] == True)
    x_test = x_test.values.astype(np.float32, copy=False)

    print("Start prediction ...")
    # num_threads > 1 will predict very slow in kernal
    model.reset_parameter({"num_threads":4})
    #sub[d] = (1 - CalibrationFactor) * clf.predict(x_test) + cal
    sub[d] = model.predict(x_test)
    
    sub.to_csv('lgb_starter.csv', index=False, float_format='%.4f')
    del x_test; gc.collect()
    print('%s done.' % d)
end = time.time()
print('time elapsed %ds' % (end - start))

Start write result ...


NameError: name 'pd' is not defined